In [1]:
import spotipy
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import re

load_dotenv('../secrets/.env')

%run useful_functions.ipynb
%run spotify_scraping.ipynb

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [2]:
shows = get_shows()
show_broadcast_details = get_show_broadcast_details()
broadcast_details = get_broadcast_details()
broadcast_playlists = get_broadcast_playlists()

master_df = (
broadcast_details
.merge(broadcast_playlists, on='Broadcast Key', how='left', suffixes=(" Broadcast Details",""))
.merge(show_broadcast_details, on='Broadcast Key', how='left', suffixes=(""," Show Details"))
.merge(shows, on='Show Key', how='left')
.sort_values(['Service Date','Broadcast Position'], ascending = [False, True])
)

tracks = get_spotify_tracks()
albums = get_spotify_albums()
artists = get_spotify_artists()

In [3]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year) == 2022) & (master_df['Show Name'] == "Radio 1's Indie Show with Jack Saunders")]

tracks_to_run = demo_df[['Track Name', 'Artist Name']].sort_values(['Artist Name','Track Name'], ascending =  True).drop_duplicates(subset=['Artist Name','Track Name'], keep='first')
tracks_to_run = tracks_to_run.merge(tracks[['Original Search Artist','Original Search Track']], how='left', left_on=['Artist Name','Track Name'], right_on=['Original Search Artist','Original Search Track'], copy=False,indicator=True)
tracks_to_run = tracks_to_run[tracks_to_run['_merge']!="both"][['Artist Name','Track Name']]
tracks_to_run

,Artist Name,Track Name
25,Automotion,Automotion


In [5]:
for i, (index, row) in enumerate(tracks_to_run.iterrows()):
    print("Iteration ",i+1," of ",len(tracks_to_run.index),":",row['Artist Name'],":", row['Track Name'])
    (temp_tracks,temp_albums,temp_artists) = get_track_info(row['Artist Name'],row['Track Name'])
    
    tracks = pd.concat([tracks,temp_tracks])
    albums = pd.concat([albums,temp_albums])
    artists = pd.concat([artists,temp_artists])
    
tracks = tracks.drop_duplicates(subset=['Original Search Artist', 'Original Search Track'], keep='last')
albums = albums.drop_duplicates(subset='Album ID', keep='last')
artists = artists.drop_duplicates(subset='Artist ID', keep='last')

Iteration  1  of  1 : Automotion : Automotion
Error with:  Automotion : Automotion


In [44]:
tracks.to_pickle("../data/spotify_tracks.pkl")
albums.to_pickle("../data/spotify_albums.pkl")
artists.to_pickle("../data/spotify_artists.pkl")